## RESULTS FROM CLUSTERING

In [1]:
# for data
from os import listdir
from os.path import isfile, join
from functools import reduce

import re
from itertools import combinations
from itertools import chain

import pandas as pd
import numpy as np

# for geospatial
import geopandas as gpd
import geojson
import rtree
import pyproj

# for viz
import folium.folium

import seaborn as sns
import matplotlib.pyplot as plt

import folium
from folium import plugins
import geoplot

%matplotlib inline

pd.set_option('display.max_rows', 1000)
sns.set_style("darkgrid", {"axes.facecolor": ".8"})

### GLOBAL VARIABLES

In [2]:
# GLOBAL VARIABLES

FINAL_DATA_PATH = '../data/final_streamlit'

NAMES_DICT = {
    
    'cadastralparcel.geojson': 'CAD_PA',
    'otherconstruction.geojson': 'O_CONS',
    'buildingpart.geojson': 'P_BU',
    'building.geojson': 'BU',
    'cadastralzoning.geojson': 'CAD_ZO',
    '28900.geojson': 'MAD',
    
    'building_points.geojson': 'BU_POINTS',
    'arturo.geojson': 'ARTURO_DF',
    'building_polygs.geojson': 'BU_POLYGONS',
    'building_parcls.geojson': 'BU_PARCELS',
    
    'bu_parcel_epsg3857.geojson' : 'BU_PARCELS',
}

In [3]:
def getting_final_geoframes(geometries_path):
    """
    INPUT:
    OUTPUT:
    """
    # List with necessary files    
    geom_bu_files = [f for f in listdir(geometries_path) if isfile(join(geometries_path, f)) and re.findall('bu', f)]   # only retrieves files names with 'building' in them
    print(f"\n-- Opening {len(geom_bu_files)} files in {geometries_path} --------------------------------------------------------------")
    
    for file in geom_bu_files:

        # CHANGE NAME OF PARCELS FILE IF CONVINIENT
        # Constructing a GeoDatFrame and giving them a name
        # Note: yield directly from gpd.read_files doesnt work like in pandas, returns constructor
        geom_file = gpd.read_file(f"{geometries_path}/{file}").set_index('ID')
        geom_file.name = NAMES_DICT[file]
        
        print(f"\t{file.split('.')[0]} \tOPENED \tMemory Usage:\t{np.round(geom_file.memory_usage().sum()/1000000, 2)} Mb \t\tShape: {geom_file.shape}")
        return geom_file


In [4]:
bu_parcel = getting_final_geoframes(FINAL_DATA_PATH)


-- Opening 1 files in ../data/final_streamlit --------------------------------------------------------------
	bu_parcel_epsg3857 	OPENED 	Memory Usage:	64.86 Mb 		Shape: (121004, 66)


In [16]:
def selecting_morfologic_cluster(gdf, entry):
    if entry == 'residence':

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87])

In [53]:
SELECTING_URBANQLITY_CLUSTER = np.sort(bu_parcel['cluster_all'].value_counts().index.tolist())
SELECTING_MORFOLOGIC_CLUSTER = np.sort(bu_parcel['cluster_build'].value_counts().index.tolist())
SELECTING_USE = bu_parcel['currentUse'].value_counts().index.tolist()

DISSOLVE_COLUMN = 'cluster_all'

bu_aux = bu_parcel.dissolve(by=DISSOLVE_COLUMN)

bu_aux['geometry'] = bu_aux['geometry'].simplify(5, preserve_topology=True)
bu_aux['value'] = bu_aux.apply(lambda x: np.round(x['value'], 3), axis = 1)

,n_BuildingUnits,n_Dwellings,nFloors_AG,nFloors_BG,area_m2c,area_m2p,train,land_use_mix,closeness_small_parks,residence_ratio,...,street_orientation,street_centrality_degree,street_centrality_eigenvector,street_centrality_betweenness,street_centrality_closeness,street_hierarchy_primary,street_hierarchy_secondary,street_hierarchy_tertiary,value,cluster_build
cluster_all,,,,,,,,,,,,,,,,,,,,,
0,1,0,2,3,2698,3152,0.0,0.000125,500.0,0.023114,...,91.155,0.00008,0.105477,0.001115,0.01917,0.0,0.0,1.0,17.598,7


In [52]:
SELECTING_URBANQLITY_CLUSTER = np.sort(bu_parcel['urbanQuality_Clusterization'].value_counts().index.tolist())
SELECTING_MORFOLOGIC_CLUSTER = np.sort(bu_parcel['morphologic_Clusterization'].value_counts().index.tolist())
SELECTING_USE = bu_parcel['currentUse'].value_counts().index.tolist()

SELECTING_USE

,n_BuildingUnits,n_Dwellings,nFloors_AG,nFloors_BG,area_m2c,area_m2p,train,land_use_mix,closeness_small_parks,residence_ratio,...,street_orientation,street_centrality_degree,street_centrality_eigenvector,street_centrality_betweenness,street_centrality_closeness,street_hierarchy_primary,street_hierarchy_secondary,street_hierarchy_tertiary,value,cluster_build
cluster_all,,,,,,,,,,,,,,,,,,,,,
0,1,0,2,3,2698,3152,0.0,0.000125,500.0,0.023114,...,91.155,0.00008,0.105477,0.001115,0.01917,0.0,0.0,1.0,17.598,7


In [5]:
aux = bu_parcel.groupby(by = 'cluster_build').mean()
aux['ocupación'] = aux['area_m2c'] /aux['area_m2p']

table = aux[['n_BuildingUnits','n_Dwellings', 'nFloors_AG', 'nFloors_BG', 'area_m2c', 'area_m2p', 'ocupación']]

In [119]:
main_cols = ['currentUse', 'nFloors_AG', 'nFloors_BG', 'area_m2c', 'area_m2p', 'anisotropicity', 'average_quality', 'average_age', 'residence_ratio', 'built_density', 'culture_density', 'leisure_density', 
             'closeness_small_parks', 'closeness_large_parks', 'office_surface', 'value', 'cluster_build']

bu_aux[(bu_aux['currentUse'] == 'residential')].groupby(by = 'cluster_build')[main_cols].mean()

,nFloors_AG,nFloors_BG,area_m2c,area_m2p,anisotropicity,average_quality,average_age,residence_ratio,built_density,culture_density,leisure_density,closeness_small_parks,closeness_large_parks,office_surface,value,cluster_build
cluster_build,,,,,,,,,,,,,,,,
0,5.000000,0.0,993.000000,240.000000,0.042886,3.475405,90.720005,0.849411,3.837496,0.000019,0.000426,500.000000,500.000000,32696.000000,27.832074,0.0
2,2.666667,0.0,427.888889,218.444444,0.020782,4.912916,108.688263,0.703508,2.484795,0.000031,0.000220,225.174161,500.000000,1299.111111,24.420184,2.0
3,4.142857,0.0,894.685714,439.657143,0.029865,5.002605,111.884386,0.557762,2.436210,0.000004,0.000142,228.984679,473.449601,1424.485714,23.811317,3.0
4,4.200000,3.0,1069.600000,348.400000,0.024304,4.789670,105.830912,0.737333,3.607492,0.000020,0.000347,200.073794,475.000000,5190.400000,24.443351,4.0
5,3.500000,3.0,1095.000000,550.500000,0.028769,4.195930,115.282700,0.671519,1.136700,0.000006,0.000027,375.000000,500.000000,4595.250000,22.916581,5.0
6,5.000000,9.0,3637.000000,1165.000000,0.029154,5.596494,117.061348,0.667381,2.692730,0.000000,0.000285,0.000000,500.000000,1912.000000,24.263681,6.0
7,5.736842,3.0,2278.578947,602.368421,0.029995,4.820464,113.338785,0.670559,3.154488,0.000006,0.000179,223.335261,456.226895,3559.868421,23.336051,7.0
9,10.000000,6.0,8607.500000,836.000000,0.036236,3.617308,93.045521,0.760209,2.891185,0.000017,0.000290,0.000000,500.000000,23269.000000,27.109236,9.0
10,8.000000,0.0,2044.000000,337.000000,0.068196,4.217271,112.352924,0.914195,2.369197,0.000058,0.000058,250.000000,500.000000,239.000000,25.398820,10.0


In [7]:
urban_indicators = ['value', 'built_density', 'residence_ratio', 'closeness_small_parks',  'commerce_surface', 'leisure_density', 'housing_surface', 
                'street_centrality_degree', 'street_hierarchy_primary', 'street_hierarchy_secondary', 'closeness_primary_roads', 'closeness_secondary_roads', 'tertiary_roads_length',
                     'average_age']

morphologic_indicators = [b'average_nFloors_AGround', 'occupation_coheficient', '']

NameError: name 'bu_aux' is not defined

In [20]:
bu_aux[['cluster_all']]

KeyError: "None of [Index(['cluster_all'], dtype='object')] are in the [columns]"

### FOLIUM ATTEMPT VRS.2 

In [22]:
import folium
from folium.plugins import Fullscreen
import branca
import branca.colormap as cm


# Color Palettes for maps
customCPal = ['#F2BF6C', #C29450'
              "#F5E6CB","#F4DCC7","#F4D3C3","#F3C9C0","#F3C0BC","#F2B6B8","#F2ADB4","#F1A3B1","#F19AAD",
              "#F090A9","#E387A6","#D57DA3","#C874A1","#BB6A9E","#AD619B","#A05798","#934E96","#854493","#783B90","#6A318D","#5D288A","#501E88","#421585","#350B82"][::-1]


LinearCMAP = cm.LinearColormap(customCPal,
                               index = [i/len(customCPal) for i in range(0, len(customCPal))])
color_palette = [LinearCMAP(i/len(bu_aux)) for i in range(0, len(bu_aux))]

# MAP

gdf_to_map = bu_aux

m = folium.Map( # 'EPSG3857' by default
    location = [40.4168, -3.7038], # MADRID
    zoom_start = 14, max_zoom = 16, min_zoom = 13,
    min_lat = 40.15, max_lat = 40.75, min_lon = -3.25, max_lon = -3.95, max_bounds=True,
    control_scale = False,
    prefer_canvas = True,
    tiles = 'CartoDB PositronNoLabels',
 )


for i,val in enumerate(gdf_to_map.index.tolist()):
    
    fillColor = color_palette[i]
    
    # enabling pop up is a killer
    
    gjson_popup = folium.features.GeoJsonTooltip(fields = ['value', 'currentUse', 'centuryOfConstr', 'nFloors_AG', 'nFloors_BG'],
                                                   aliases = ['Voting_Mean', 'Main_Use', 'Construction_Age', 'numFloors_aboveGr', 'numFloors_belowGr'],
                                                   localize = True,
                                                   sticky = True)
    
    gjson = folium.GeoJson(gdf_to_map[gdf_to_map.index == val],
                   name = f"cluster_{val}",
                   tooltip = gjson_popup,
                   style_function = lambda x, fillColor=fillColor: {
                                                       'fillColor': fillColor,
                                                       'color' : '#000000',
                                                       'weight': 0.1,
                                                       'fillOpacity': 0.95}
              ).add_to(m)


    
Fullscreen().add_to(m)
m.save('tryout.html')